# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,8.76,80,100,3.60,US,1735615945
1,1,conde,-7.2597,-34.9075,25.70,83,40,2.57,BR,1735615946
2,2,invercargill,-46.4000,168.3500,16.68,71,14,2.24,NZ,1735615947
3,3,jiuquan,39.7432,98.5174,-1.11,24,22,1.98,CN,1735615948
4,4,ihosy,-22.4000,46.1167,21.45,85,79,2.70,MG,1735615949


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color= "City",
    scale=1
    
)
# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather =  city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & 
                                  (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,tura,25.5198,90.2201,22.04,44,0,1.40,IN,1735615988
55,55,sherpur,25.0188,90.0175,21.92,52,0,4.32,BD,1735616009
71,71,santana,-0.0583,-51.1817,25.10,94,0,2.06,BR,1735616027
158,158,brisas de zicatela,15.8369,-97.0419,25.29,88,0,1.03,MX,1735616127
225,225,yendi,9.4427,-0.0099,24.88,23,0,1.22,GH,1735616203
228,228,bonthe,7.5264,-12.5050,23.17,91,0,1.95,SL,1735616207
406,406,saint-pierre,-21.3393,55.4781,26.82,61,0,2.06,RE,1735616412
545,545,kodinar,20.7903,70.7031,22.98,45,0,2.56,IN,1735616570


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,tura,IN,25.5198,90.2201,44,
55,sherpur,BD,25.0188,90.0175,52,
71,santana,BR,-0.0583,-51.1817,94,
158,brisas de zicatela,MX,15.8369,-97.0419,88,
225,yendi,GH,9.4427,-0.0099,23,
228,bonthe,SL,7.5264,-12.5050,91,
406,saint-pierre,RE,-21.3393,55.4781,61,
545,kodinar,IN,20.7903,70.7031,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories" : "accommodation.hotel",
    "limit": 1,
    "api_key": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"]= f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
sherpur - nearest hotel: No hotel found
santana - nearest hotel: Hotel Teodoro
brisas de zicatela - nearest hotel: Casa de Olas
yendi - nearest hotel: White House Lodge
bonthe - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
kodinar - nearest hotel: Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
36,tura,IN,25.5198,90.2201,44,Hotel Polo Orchid
55,sherpur,BD,25.0188,90.0175,52,No hotel found
71,santana,BR,-0.0583,-51.1817,94,Hotel Teodoro
158,brisas de zicatela,MX,15.8369,-97.0419,88,Casa de Olas
225,yendi,GH,9.4427,-0.0099,23,White House Lodge
228,bonthe,SL,7.5264,-12.5050,91,No hotel found
406,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
545,kodinar,IN,20.7903,70.7031,45,Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE